# Group operations - split-apply-combine
- grouping key can take many forms:
 - index or columns
 - keys do not have to be of same type
 - a list/array same length as grouping axis
 - a dictionary/Series corresponding to values on grouping axis
 - a function to be invoked on the axis or individual labels on the index

In [57]:
import pandas as pd
import numpy as np

In [91]:

np.random.seed(42)
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
"key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
"data1" : np.random.standard_normal(7),
"data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.496714,0.767435
1,a,2,-0.138264,-0.469474
2,None,1,0.647689,0.542560
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730
5,a,<NA>,-0.234137,0.241962
6,None,1,1.579213,-1.913280


In [93]:
# df = df.set_index(keys="key1")
# df.index.name = "key1"
df.groupby(by="key1", dropna=False).mean()

,key2,data1,data2
key1,,,
a,1.5,0.041438,0.179974
b,1.5,0.644438,-0.464574
NaN,1.0,1.113451,-0.685360


In [71]:
grouped = df["data1"].groupby(df["key1"])
grouped.mean()

key1
None    1.113451
a       0.041438
b       0.644438
Name: data1, dtype: float64

In [73]:
mean_df = df.groupby(by=["key1", "key2"]).mean()

In [90]:
mean_df

data     data1               data2          
key2         1         2         1         2
key1                                        
None  1.113451       NaN -0.685360       NaN
a     0.496714 -0.138264  0.767435 -0.469474
b    -0.234153  1.523030 -0.465730 -0.463418

In [78]:
mean_df = mean_df.unstack()

In [79]:
mean_df.columns.nlevels

2

In [80]:
mean_df

data1               data2          
key2         1         2         1         2
key1                                        
None  1.113451       NaN -0.685360       NaN
a     0.496714 -0.138264  0.767435 -0.469474
b    -0.234153  1.523030 -0.465730 -0.463418

In [86]:
mean_df.columns.names

FrozenList([None, 'key2'])

In [87]:
mean_df.columns.names = ["data", "key2"]
mean_df

data     data1               data2          
key2         1         2         1         2
key1                                        
None  1.113451       NaN -0.685360       NaN
a     0.496714 -0.138264  0.767435 -0.469474
b    -0.234153  1.523030 -0.465730 -0.463418

In [88]:
mean_df.swaplevel(i="data", j="key2", axis=1)

key2,1,2,1,2
data,data1,data1,data2,data2
key1,,,,
None,1.113451,NaN,-0.685360,NaN
a,0.496714,-0.138264,0.767435,-0.469474
b,-0.234153,1.523030,-0.465730,-0.463418


In [89]:
df.groupby(["key1", "key2"]).size()

key1  key2
None  1       2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

- `count` computes number of non-null values in group

In [98]:
df.groupby(["key1"], dropna=False).count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2
NaN,2,2,2


In [100]:
for key1, group in df.groupby("key1"):
    print(key1)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.496714  0.767435
1    a     2 -0.138264 -0.469474
5    a  <NA> -0.234137  0.241962
b
  key1  key2     data1     data2
3    b     2  1.523030 -0.463418
4    b     1 -0.234153 -0.465730


In [101]:
for (key1, key2), group in df.groupby(["key1", "key2"]):
    print(key1, key2)
    print(group)

a 1
  key1  key2     data1     data2
0    a     1  0.496714  0.767435
a 2
  key1  key2     data1     data2
1    a     2 -0.138264 -0.469474
b 1
  key1  key2     data1    data2
4    b     1 -0.234153 -0.46573
b 2
  key1  key2    data1     data2
3    b     2  1.52303 -0.463418


In [102]:
pieces = {name: group for name, group in df.groupby(["key1", "key2"])}
pieces

{('a',
  1):   key1  key2     data1     data2
 0    a     1  0.496714  0.767435,
 ('a',
  2):   key1  key2     data1     data2
 1    a     2 -0.138264 -0.469474,
 ('b',
  1):   key1  key2     data1    data2
 4    b     1 -0.234153 -0.46573,
 ('b',
  2):   key1  key2    data1     data2
 3    b     2  1.52303 -0.463418}

- grouping on `columns` axis

In [106]:
grouped_cols = df.groupby({"key1": "key", "key2": "key", "data1": "data", "data2": "data"}, axis=1)
for name, group in grouped_cols:
    print(name)
    print(group)

data
      data1     data2
0  0.496714  0.767435
1 -0.138264 -0.469474
2  0.647689  0.542560
3  1.523030 -0.463418
4 -0.234153 -0.465730
5 -0.234137  0.241962
6  1.579213 -1.913280
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [107]:
s_grouped = df.groupby(["key1", "key2"])["data1"]

In [111]:
print(*s_grouped)

(('a', 1), 0    0.496714
Name: data1, dtype: float64) (('a', 2), 1   -0.138264
Name: data1, dtype: float64) (('b', 1), 4   -0.234153
Name: data1, dtype: float64) (('b', 2), 3    1.52303
Name: data1, dtype: float64)


 - grouping by `dictionary` and `Series`

In [112]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
columns=["a", "b", "c", "d", "e"],
index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
 

In [114]:
people

,a,b,c,d,e
Joe,-1.724918,-0.562288,-1.012831,0.314247,-0.908024
Steve,-1.412304,1.465649,-0.225776,0.067528,-1.424748
Wanda,-0.544383,NaN,NaN,0.375698,-0.600639
Jill,-0.291694,-0.601707,1.852278,-0.013497,-1.057711
Trey,0.822545,-1.220844,0.208864,-1.959670,-1.328186


In [117]:
grp_mapping = {"a": "red",
"b": "red", 
"c": "orange",
"d": "orange",
"e": "red",
"f": "green" } # unused group mappings are ok
by_col = people.groupby(grp_mapping, axis=1, dropna=False)
by_col.mean()

,orange,red
Joe,-0.349292,-1.065076
Steve,-0.079124,-0.457134
Wanda,0.375698,-0.572511
Jill,0.919390,-0.650370
Trey,-0.875403,-0.575495


In [118]:
map_series = pd.Series(grp_mapping)
map_series

a       red
b       red
c    orange
d    orange
e       red
f     green
dtype: object

In [120]:

by_col = people.groupby(grp_mapping, axis=1, dropna=False)
by_col.mean()
by_col.count()

,orange,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


- by `functions`

In [122]:
# group over all people with same length of names
people.groupby(len).count() # computes len on each index value and return value used as index

,a,b,c,d,e
3,1,1,1,1,1
4,2,2,2,2,2
5,2,1,1,2,2


- mix arrays with functions

In [123]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).count()

,,a,b,c,d,e
3,one,1,1,1,1,1
4,two,2,2,2,2,2
5,one,2,1,1,2,2


In [124]:
people

,a,b,c,d,e
Joe,-1.724918,-0.562288,-1.012831,0.314247,-0.908024
Steve,-1.412304,1.465649,-0.225776,0.067528,-1.424748
Wanda,-0.544383,NaN,NaN,0.375698,-0.600639
Jill,-0.291694,-0.601707,1.852278,-0.013497,-1.057711
Trey,0.822545,-1.220844,0.208864,-1.959670,-1.328186


- groupby `index levels`

In [125]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
[1, 3, 5, 1, 3]],
names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns = columns)
hier_df


cty          US                            JP          
tenor         1         3         5         1         3
0      0.196861  0.738467  0.171368 -0.115648 -0.301104
1     -1.478522 -0.719844 -0.460639  1.057122  0.343618
2     -1.763040  0.324084 -0.385082 -0.676922  0.611676
3      1.031000  0.931280 -0.839218 -0.309212  0.331263

In [130]:
hier_df.groupby(level=1, axis="columns").mean()

tenor,1,3,5
0,0.040606,0.218681,0.171368
1,-0.210700,-0.188113,-0.460639
2,-1.219981,0.467880,-0.385082
3,0.360894,0.631272,-0.839218


# Data Aggregation

In [138]:
df

,key1,key2,data1,data2
0,a,1,0.496714,0.767435
1,a,2,-0.138264,-0.469474
2,None,1,0.647689,0.542560
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730
5,a,<NA>,-0.234137,0.241962
6,None,1,1.579213,-1.913280


In [141]:
grouped = df.groupby("key1")

def max_plus1(arr):
    return arr.max() + 1

grouped.apply(max_plus1)

,key2,data1,data2
key1,,,
a,3.0,1.496714,1.767435
b,3.0,2.523030,0.536582


In [142]:
grouped.agg(max_plus1)

,key2,data1,data2
key1,,,
a,3,1.496714,1.767435
b,3,2.523030,0.536582


In [143]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.041438  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.644438  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.179225  0.496714   3.0  0.179974  0.620780 -0.469474 -0.113756   
b     1.083734  1.523030   2.0 -0.464574  0.001635 -0.465730 -0.465152   

                                    
           50%       75%       max  
key1                                
a     0.241962  0.504699  0.767435  
b    -0.464574 -0.463996 -0.463418  

[2 rows x 24 columns]

In [144]:
grouped.mean()

,key2,data1,data2
key1,,,
a,1.5,0.041438,0.179974
b,1.5,0.644438,-0.464574


# Column-wise and Multiple function application

In [173]:
tips = pd.read_csv("examples/tips.csv")
tips["tip_pct"] = tips["tip"]/ tips["total_bill"]
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [188]:
tips_grp = tips.groupby(["smoker", "day"])
tips_grp.describe()

total_bill                                                        \
                 count       mean        std    min     25%     50%      75%   
smoker day                                                                     
No     Fri         4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
       Sat        45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
       Sun        57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
       Thur       45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
Yes    Fri        15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
       Sat        42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
       Sun        19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
       Thur       17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

                     tip            ...  size      tip_pct            \
               max count      mean  ...   75%  max   count      mean   
smoker day                          ...                                
No     Fri   22.75   4.0  2.812500  ...  2.25  3.0     4.0  0.151650   
       Sat   48.33  45.0  3.102889  ...  3.00  4.0    45.0  0.158048   
       Sun   48.17  57.0  3.167895  ...  4.00  6.0    57.0  0.160113   
       Thur  41.19  45.0  2.673778  ...  2.00  6.0    45.0  0.160298   
Yes    Fri   40.17  15.0  2.714000  ...  2.00  4.0    15.0  0.174783   
       Sat   50.81  42.0  2.875476  ...  3.00  5.0    42.0  0.147906   
       Sun   45.35  19.0  3.516842  ...  3.00  5.0    19.0  0.187250   
       Thur  43.11  17.0  3.030000  ...  2.00  4.0    17.0  0.163863   

                                                                         
                  std       min       25%       50%       75%       max  
smoker day                                                               
No     Fri   0.028123  0.120385  0.137239  0.149241  0.163652  0.187735  
       Sat   0.039767  0.056797  0.136240  0.150152  0.183915  0.291990  
       Sun   0.042347  0.059447  0.139780  0.161665  0.185185  0.252672  
       Thur  0.038774  0.072961  0.137741  0.153492  0.184843  0.266312  
Yes    Fri   0.051293  0.103555  0.133739  0.173913  0.209240  0.263480  
       Sat   0.061375  0.035638  0.091797  0.153624  0.190502  0.325733  
       Sun   0.154134  0.065660  0.097723  0.138122  0.215325  0.710345  
       Thur  0.039389  0.090014  0.148038  0.153846  0.194837  0.241255  

[8 rows x 32 columns]

In [147]:
tips_grp.agg(["mean", "std", max_plus1])

/var/folders/nx/j3jrdkw97q1gxc0nkyjz0sg80000gn/T/ipykernel_11833/2761459862.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  tips_grp.agg(["mean", "std", max_plus1])


total_bill                            tip                      \
                  mean        std max_plus1      mean       std max_plus1   
smoker day                                                                  
No     Fri   18.420000   5.059282     23.75  2.812500  0.898494      4.50   
       Sat   19.661778   8.939181     49.33  3.102889  1.642088     10.00   
       Sun   20.506667   8.130189     49.17  3.167895  1.224785      7.00   
       Thur  17.113111   7.721728     42.19  2.673778  1.282964      7.70   
Yes    Fri   16.813333   9.086388     41.17  2.714000  1.077668      5.73   
       Sat   21.276667  10.069138     51.81  2.875476  1.630580     11.00   
       Sun   24.120000  10.442511     46.35  3.516842  1.261151      7.50   
       Thur  19.190588   8.355149     44.11  3.030000  1.113491      6.00   

                 size                      
                 mean       std max_plus1  
smoker day                                 
No     Fri   2.250000  0.500000         4  
       Sat   2.555556  0.784960         5  
       Sun   2.929825  1.032674         7  
       Thur  2.488889  1.179796         7  
Yes    Fri   2.066667  0.593617         5  
       Sat   2.476190  0.862161         6  
       Sun   2.578947  0.901591         6  
       Thur  2.352941  0.701888         5

In [152]:
tips_grp.agg([("average", "mean"), ("std-dev", lambda x: x.std())])

/var/folders/nx/j3jrdkw97q1gxc0nkyjz0sg80000gn/T/ipykernel_11833/1390994588.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  tips_grp.agg([("average", "mean"), ("std-dev", lambda x: x.std())])


total_bill                  tip                size          
               average    std-dev   average   std-dev   average   std-dev
smoker day                                                               
No     Fri   18.420000   5.059282  2.812500  0.898494  2.250000  0.500000
       Sat   19.661778   8.939181  3.102889  1.642088  2.555556  0.784960
       Sun   20.506667   8.130189  3.167895  1.224785  2.929825  1.032674
       Thur  17.113111   7.721728  2.673778  1.282964  2.488889  1.179796
Yes    Fri   16.813333   9.086388  2.714000  1.077668  2.066667  0.593617
       Sat   21.276667  10.069138  2.875476  1.630580  2.476190  0.862161
       Sun   24.120000  10.442511  3.516842  1.261151  2.578947  0.901591
       Thur  19.190588   8.355149  3.030000  1.113491  2.352941  0.701888

- customs function for each column

In [160]:
tips_grp.agg({"total_bill": [("bill_max", "max"), "mean"], "time": ["count", np.unique], "tip": ["mean", "min"]})

total_bill             time                        tip      
              bill_max       mean count           unique      mean   min
smoker day                                                              
No     Fri       22.75  18.420000     4  [Dinner, Lunch]  2.812500  1.50
       Sat       48.33  19.661778    45         [Dinner]  3.102889  1.00
       Sun       48.17  20.506667    57         [Dinner]  3.167895  1.01
       Thur      41.19  17.113111    45  [Dinner, Lunch]  2.673778  1.25
Yes    Fri       40.17  16.813333    15  [Dinner, Lunch]  2.714000  1.00
       Sat       50.81  21.276667    42         [Dinner]  2.875476  1.00
       Sun       45.35  24.120000    19         [Dinner]  3.516842  1.50
       Thur      43.11  19.190588    17          [Lunch]  3.030000  2.00

In [182]:
# disable index
tips.groupby(["day", "smoker"], group_keys=False)[["total_bill", "tip"]].agg(["mean", "std"]) 

total_bill                  tip          
                  mean        std      mean       std
day  smoker                                          
Fri  No      18.420000   5.059282  2.812500  0.898494
     Yes     16.813333   9.086388  2.714000  1.077668
Sat  No      19.661778   8.939181  3.102889  1.642088
     Yes     21.276667  10.069138  2.875476  1.630580
Sun  No      20.506667   8.130189  3.167895  1.224785
     Yes     24.120000  10.442511  3.516842  1.261151
Thur No      17.113111   7.721728  2.673778  1.282964
     Yes     19.190588   8.355149  3.030000  1.113491

# Split-Apply-Combine

In [174]:
def top5(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

tips.groupby("smoker").apply(top5)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [177]:
tips.groupby("smoker").apply(top5, n=2, column="total_bill")

total_bill    tip smoker  day    time  size   tip_pct
smoker                                                           
No     212       48.33   9.00     No  Sat  Dinner     4  0.186220
       59        48.27   6.73     No  Sat  Dinner     4  0.139424
Yes    170       50.81  10.00    Yes  Sat  Dinner     3  0.196812
       182       45.35   3.50    Yes  Sun  Dinner     3  0.077178

In [185]:

tips.groupby("smoker", group_keys=False, as_index=False).apply(top5, n=2, column="total_bill")

,total_bill,tip,smoker,day,time,size,tip_pct
212,48.33,9.00,No,Sat,Dinner,4,0.186220
59,48.27,6.73,No,Sat,Dinner,4,0.139424
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
182,45.35,3.50,Yes,Sun,Dinner,3,0.077178


- quantile & bucket analysis `pd.cut`